# Gather

#### Download the file 'twitter-archive-enhanced.csv' from the link [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)

In [25]:
import pandas as pd

In [26]:
# Load data frame with twitter-archive-enhanced.csv
tweet_archive_data = pd.read_csv('twitter-archive-enhanced.csv')
tweet_archive_data.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


#### Download the image_predictions.tsv programmatically from this [url](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)

In [27]:
import requests
import os

In [4]:
# download the file from the below url
url ='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [5]:
with open(url.split('/')[-1], mode='wb') as file:

    file.write(response.content)

In [6]:
os.listdir(os.curdir)

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'act_report.html',
 'act_report.ipynb',
 'image-predictions.tsv',
 'README.md',
 'tweet_archive_data_clean.csv',
 'tweet_count_data_clean.csv',
 'tweet_image_data_clean.csv',
 'tweet_json.txt',
 'twitter-archive-enhanced.csv',
 'twitter_archive_master.csv',
 'wrangle_act.html',
 'wrangle_act.ipynb']

In [28]:
# Load dataframe with image-predictions.tsv 
tweet_image_data = pd.read_csv('image-predictions.tsv', sep = '\t')
tweet_image_data.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


#### Gather tweet and retweet counts  for the tweet ids in the archive dataset

In [8]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# Keys are hidden to comply with Twitter's API terms and conditions
# Two rerun this replace the hidden with your own keys after subscribing to twitter developer account
import tweepy

consumer_key = 'Hidden-Key'
consumer_secret = 'Hidden-Key'
access_token = 'Hidden-Key'
access_secret = 'Hidden-Key'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [29]:
import json
from timeit import default_timer as timer

In [30]:
# Tweet ids in the twitter-archive-enhanced.csv file
tweet_ids = tweet_archive_data.tweet_id.values
len(tweet_ids)

2356

In [11]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        # print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            # print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            # print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1891.9851507429994
{888202515573088257: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 873697596434513921: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 872668790621863937: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 869988702071779329: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 866816280283807744: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 861769973181624320: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 845459076796616705: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 842892208864923648: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 837012587749474308: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 827228250799742977: TweepError([{'code': 144, 'message': 'No status found with that ID.'}],), 812747805718642688: TweepError([{'code':

In [12]:
# how many twitter ids the data retreival failed
len(fails_dict)

16

In [31]:
# read json data from tweet_json.txt and load into a dataframe
tweet_count_list = []
with open('tweet_json.txt') as json_file:
    for line in json_file:
        json_data = json.loads(line)
        tweet_id = json_data['id']
        retweet_count = json_data['retweet_count']
        favorite_count = json_data['favorite_count']
        tweet_count_list.append({'tweet_id': tweet_id,
                        'retweet_count': int(retweet_count),
                        'favorite_count': int(favorite_count)
                        })



tweet_count_data = pd.DataFrame(tweet_count_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])
        

In [32]:
tweet_count_data.head(2)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8257,37848
1,892177421306343426,6101,32523


# Assess

### Visual Assessment
#### For tweet_archive_data

In [33]:
tweet_archive_data

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [34]:
tweet_archive_data.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [35]:
tweet_archive_data.tail(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [37]:
tweet_archive_data.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
458,817908911860748288,NaN,NaN,2017-01-08 01:40:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Looks like he went cross-eyed trying way too h...,NaN,NaN,NaN,https://twitter.com/micahgrimes/status/8179020...,12,10,None,None,None,None,None
2020,672095186491711488,NaN,NaN,2015-12-02 16:49:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tuco. That's the toast that killed his...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672095186...,9,10,Tuco,None,None,None,None
1664,682788441537560576,NaN,NaN,2016-01-01 05:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy New Year from your fav holiday squad! 🎉 ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682788441...,12,10,None,None,None,pupper,None
916,756998049151549440,NaN,NaN,2016-07-23 23:42:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oliver. He's an English Creamschnitzel...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/756998049...,11,10,Oliver,None,None,None,None
1343,704761120771465216,NaN,NaN,2016-03-01 20:11:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pupper killed this great white in an epic...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704761120...,13,10,None,None,None,pupper,None


##### Upon just visual observation we can see that
##### 1. we have 17 observations in this data set
##### 2. There are NaN values in in_reply_to_status_id , in_reply_to_user_id , retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns. 
##### 3. Column "name" seem to represent "None" values and incorrect values like " a,the, an, etc"
##### 4. Some records have a denominator value different from 10
##### 5. Last four columns seem to be holding dog breed type and this structurally untidy. 

### Programmatic Assessment
#### For tweet_archive_data

In [10]:
tweet_archive_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

##### tweet_archive_data has 17 columns and 2356 observations, tweet_id seems to be the unique Id
##### in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id , retweeted_status_user_id, retweeted_status_timestamp columns has very few non null values.

##### timestamp column is not represented as date.

##### Let's see if there are any duplicate records based on tweet_id

In [15]:
tweet_archive_data[tweet_archive_data.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


#### No duplicates in the table for tweet_id

In [38]:
tweet_archive_data.tweet_id.nunique()

2356

##### tweet_id is the unique id and there are 2356 unique tweet_ids for 2356 observations in the tweet_archive_data 


##### Let's take a closer look at columns where we have majority NaN Values

In [19]:
tweet_archive_data.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

##### in_reply_to_status_id  has 2278 NaN
##### in_reply_to_user_id has 2278 NaN
##### retweeted_status_id has 2175 NaN
##### retweeted_status_user_id 2175 NaN
##### retweeted_status_timestamp 2175 NaN
##### Since these columns have majority values as Null these columns and we currently do  not have a way to find the missing values we will be  deleting these columns from the data frame.


##### Let's take a closer look at the values populated in the "name" column

In [21]:
tweet_archive_data.name.sample(2)

3       Darla
1132     None
Name: name, dtype: object

In [22]:
tweet_archive_data.name.value_counts()

None            745
a                55
Charlie          12
Lucy             11
Cooper           11
Oliver           11
Tucker           10
Lola             10
Penny            10
Bo                9
Winston           9
the               8
Sadie             8
Daisy             7
Bailey            7
Toby              7
Buddy             7
an                7
Jack              6
Jax               6
Oscar             6
Leo               6
Koda              6
Dave              6
Scout             6
Rusty             6
Milo              6
Bella             6
Stanley           6
Oakley            5
               ... 
Cecil             1
Yukon             1
Ziva              1
Sonny             1
Enchilada         1
Burt              1
Asher             1
Jazz              1
Strudel           1
Taco              1
Lance             1
Brutus            1
Alexanderson      1
Cannon            1
Pupcasso          1
Farfle            1
Boston            1
Kallie            1
Ike               1


##### Some of the values in the name column are "None","the","such" ,"a" and "an". These dont look like real names of the dogs. Majority of the rows ( 745) have None populated. Does this mean these rows dont have a Dog name available and data is entered as None.

##### All the valid values for the name are starting with Uppercase letter.
##### All the invalid values are starting with lowercase
##### let's take a closer look at the invalid name values and their counts

In [23]:
tweet_archive_data.loc[tweet_archive_data['name'].str.islower()].name.value_counts()

a               55
the              8
an               7
very             5
quite            4
just             4
one              4
getting          2
mad              2
actually         2
not              2
unacceptable     1
light            1
all              1
such             1
officially       1
life             1
space            1
incredibly       1
this             1
by               1
my               1
infuriating      1
old              1
his              1
Name: name, dtype: int64

In [24]:
tweet_archive_data.loc[tweet_archive_data['name'].str.islower()].name.count()

109

##### There are 109 rows that have invalid values for name
##### There are 745 rows that have name value as None


##### Let's  see where the tweets are originated from

In [25]:
tweet_archive_data.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

##### Looks like the majority of the tweets are originated from an iphone

##### Let's look at the tweet text data

In [46]:
tweet_archive_data.text.nunique()

2356

In [48]:
tweet_archive_data.text.value_counts

<bound method IndexOpsMixin.value_counts of 0       This is Phineas. He's a mystical boy. Only eve...
1       This is Tilly. She's just checking pup on you....
2       This is Archie. He is a rare Norwegian Pouncin...
3       This is Darla. She commenced a snooze mid meal...
4       This is Franklin. He would like you to stop ca...
5       Here we have a majestic great white breaching ...
6       Meet Jax. He enjoys ice cream so much he gets ...
7       When you watch your owner call another dog a g...
8       This is Zoey. She doesn't want to be one of th...
9       This is Cassie. She is a college pup. Studying...
10      This is Koda. He is a South Australian decksha...
11      This is Bruno. He is a service shark. Only get...
12      Here's a puppo that seems to be on the fence a...
13      This is Ted. He does his best. Sometimes that'...
14      This is Stuart. He's sporting his favorite fan...
15      This is Oliver. You're witnessing one of his m...
16      This is Jim. He foun

In [51]:
tweet_archive_data[['name','text']]

,name,text
0,Phineas,This is Phineas. He's a mystical boy. Only eve...
1,Tilly,This is Tilly. She's just checking pup on you....
2,Archie,This is Archie. He is a rare Norwegian Pouncin...
3,Darla,This is Darla. She commenced a snooze mid meal...
4,Franklin,This is Franklin. He would like you to stop ca...
5,None,Here we have a majestic great white breaching ...
6,Jax,Meet Jax. He enjoys ice cream so much he gets ...
7,None,When you watch your owner call another dog a g...
8,Zoey,This is Zoey. She doesn't want to be one of th...
9,Cassie,This is Cassie. She is a college pup. Studying...


##### It looks like name is extracted from the text value.

#### let's see sample records for the text value where the name is populated a None

In [55]:
tweet_archive_data.query('name=="None"')[['text']].sample(5)

,text
1949,When you're having a blast and remember tomorr...
839,I don't know any of the backstory behind this ...
923,Here's a frustrated pupper attempting to escap...
1237,Reminder that we made our first set of sticker...
1445,Oh my god 10/10 for every little hot dog pupper


##### It looks like the text does not contain the name of the dog, hence the parser couldnt not pick up any value for the dog name

##### Let's take a look at expanded_urls data

In [59]:
tweet_archive_data.expanded_urls.sample(5)

1522    https://twitter.com/dog_rates/status/690649993...
378     https://twitter.com/dog_rates/status/827933404...
744     https://twitter.com/dog_rates/status/780192070...
591     https://twitter.com/dog_rates/status/799063482...
922     https://twitter.com/dog_rates/status/756275833...
Name: expanded_urls, dtype: object

In [58]:
tweet_archive_data.expanded_urls.nunique()

2218

##### There are 2218 unique expanded urls!

In [60]:
tweet_archive_data.expanded_urls.value_counts()

https://twitter.com/dog_rates/status/694669722378485760/photo/1,https://twitter.com/dog_rates/status/694669722378485760/photo/1                                                                                                                                                                                                                                                                    2
https://twitter.com/dog_rates/status/698195409219559425/photo/1                                                                                                                                                                                                                                                                                                                                    2
https://twitter.com/dog_rates/status/667182792070062081/photo/1                                                                                                                                                               

#### Let's look at the rating_denominator and rating_numerator values

In [29]:
tweet_archive_data.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

##### Majority of the data , 2333 rows have a valid denominator of 10

In [30]:
tweet_archive_data.query("rating_denominator != 10")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


##### There are  23 rows where rating_denominator is not equal to 10

##### Lets take a closer look at the rating_numerator

In [31]:
tweet_archive_data.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

#####  According to WeRateDogs,dog ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because ["they're good dogs Brent."](https://knowyourmeme.com/memes/theyre-good-dogs-brent) WeRateDogs has over 4 million followers and has received international media coverage.

In [32]:
tweet_archive_data.query("rating_numerator > 14").sort_values('rating_numerator')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
285,838916489579200512,NaN,NaN,2017-03-07 00:57:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @KibaDva: I collected all the good dogs!! 1...,8.389060e+17,8.117408e+08,2017-03-07 00:15:46 +0000,https://twitter.com/KibaDva/status/83890598062...,15,10,None,None,None,None,None
291,838085839343206401,8.380855e+17,2.894131e+09,2017-03-04 17:56:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@bragg6of8 @Andy_Pace_ we are still looking fo...,NaN,NaN,NaN,NaN,15,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,20,16,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
1712,680494726643068929,NaN,NaN,2015-12-25 21:06:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have uncovered an entire battalion of ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680494726...,26,10,None,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697463031...,44,40,None,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


##### There are 28 rows with rating_numerator greater than 14 ..... are these exceeding the ratings for "They are good dog brent" these may be erroneous.

##### Lets take a look at the rows where rating_numerator is greater than 14 and rating_denominator is not equal to 10

In [33]:
tweet_archive_data.query("rating_numerator > 14 & rating_denominator != 10").sort_values('rating_numerator')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,20,16,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697463031...,44,40,None,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845...,60,50,a,None,None,None,None
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710658690...,80,80,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
1843,675853064436391936,NaN,NaN,2015-12-13 01:41:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have an entire platoon of puppers. Tot...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675853064...,88,80,None,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None


#####  There are 16 records that have ratings_numerator greater than 14 and ratings_denominator not equal to 10
##### These records definitely look errorneous as the ratings do not make any sense

##### Let's take a look at the last four columns of tweet_archive_data dataframe

In [34]:
tweet_archive_data[['doggo','floofer','pupper','puppo']].head(10)

,doggo,floofer,pupper,puppo
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,doggo,None,None,None


##### Looks like these four columns represent data for dog breed , which can be represented in one column called dog_breed

### Visual Assessment
#### For tweet_image_data

In [75]:
tweet_image_data

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


##### Based on just visual assessmet we can see that there are 12 observations in this table and it holds the images url for the images in each tweet and the dog breed predictions using the image recognition software. It looks like this table is pretty clean, no NaN values found so far. Although the column names are not descriptive.

### Programmatic Assessment
#### For tweet_image_date

In [76]:
tweet_image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


###### There are 12 columns and 2075 observations
###### Column names are not very descriptive.
######  None of the columns have NaN values

###### lets see if there are any duplicate tweet_ids

In [37]:
tweet_image_data[tweet_image_data.tweet_id.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


###### There are no duplicate tweet_ids so these are all unique tweets or retweets may be
###### Lets see if there are duplicates in image urls, i.e if a image is used my multiple tweets.

In [38]:
tweet_image_data[tweet_image_data.jpg_url.duplicated(keep=False)].sort_values('jpg_url')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
480,675354435921575936,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1864,842892208864923648,https://pbs.twimg.com/ext_tw_video_thumb/80710...,1,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True
1641,807106840509214720,https://pbs.twimg.com/ext_tw_video_thumb/80710...,1,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True
1703,817181837579653120,https://pbs.twimg.com/ext_tw_video_thumb/81596...,1,Tibetan_mastiff,0.506312,True,Tibetan_terrier,0.295690,True,otterhound,0.036251,True
1691,815966073409433600,https://pbs.twimg.com/ext_tw_video_thumb/81596...,1,Tibetan_mastiff,0.506312,True,Tibetan_terrier,0.295690,True,otterhound,0.036251,True
1705,817423860136083457,https://pbs.twimg.com/ext_tw_video_thumb/81742...,1,ice_bear,0.336200,False,Samoyed,0.201358,True,Eskimo_dog,0.186789,True
1858,841833993020538882,https://pbs.twimg.com/ext_tw_video_thumb/81742...,1,ice_bear,0.336200,False,Samoyed,0.201358,True,Eskimo_dog,0.186789,True
1715,819004803107983360,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,1,standard_poodle,0.351308,True,toy_poodle,0.271929,True,Tibetan_terrier,0.094759,True
1718,819015337530290176,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,1,standard_poodle,0.351308,True,toy_poodle,0.271929,True,Tibetan_terrier,0.094759,True


###### Interesting to see image url is shared by more than one tweet, what does this mean?
###### Since image predictions data is created for the twitter ids in tweet_archive_data, lets take a look at the tweet archive data for a sample of these tweets sharing the same image url.

In [39]:
tweet_archive_data.query("tweet_id in (675354435921575936,752309394570878976)")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
949,752309394570878976,NaN,NaN,2016-07-11 01:11:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Everyone needs to watch this. 1...,6.753544e+17,4.196984e+09,2015-12-11 16:40:19 +0000,https://twitter.com/dog_rates/status/675354435...,13,10,None,None,None,None,None
1865,675354435921575936,NaN,NaN,2015-12-11 16:40:19 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Everyone needs to watch this. 13/10 https://t....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675354435...,13,10,None,None,None,None,None


In [40]:
tweet_archive_data.query("tweet_id in (711694788429553666,761371037149827077)")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
872,761371037149827077,NaN,NaN,2016-08-05 01:19:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Oh. My. God. 13/10 magical af h...,7.116948e+17,4.196984e+09,2016-03-20 23:23:54 +0000,https://twitter.com/dog_rates/status/711694788...,13,10,None,None,None,None,None
1246,711694788429553666,NaN,NaN,2016-03-20 23:23:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Oh. My. God. 13/10 magical af https://t.co/Ezu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/711694788...,13,10,None,None,None,None,None


###### If you look at the retweeted_status_id , it holds the original tweet_id if it is a retweet. This is one way to recognize retweets. 

### Visual Assessment
#### For tweet_count_data

In [77]:
tweet_count_data

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8257,37848
1,892177421306343426,6101,32523
2,891815181378084864,4039,24483
3,891689557279858688,8400,41202
4,891327558926688256,9106,39378
5,891087950875897856,3022,19795
6,890971913173991426,1999,11570
7,890729181411237888,18341,63867
8,890609185150312448,4157,27212
9,890240255349198849,7172,31193


##### Based on visual assement we can see that this data set consists for three observations tweet_id, retweet_count and favorite_count
##### Looks like there aren't any NaN values

### Programmatic Assessment
#### For tweet_count_data

In [78]:
tweet_count_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


###### There are 2340 observations and three columns. There are no NaN values

## Quality

#### twitter-archive-enhanced.csv

###### 1. in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id , retweeted_status_user_id, retweeted_status_timestamp columns have majority null values.
###### 2. retweeted_status_id  is populated for retweets. To prepare a simple dataset to join with tweet counts lets remove the retweets.
###### 3. in_reply_to_status_id is populated for tweet replies.Lets remove these tweet replies.
###### 4. name column has majority of the rows ( 745) have None populated.There are also 109 rows that have invalid values for name with values like ( 'the','a','an','such' etc).
###### 5. Invalid values in ratings numerator and rating denominator. There are 16 records that have ratings numerator greater than 14 and ratings denominator not equal to 10.
###### 6. timestamp column is not in date type.
###### 7. When one of the columns in columns doggo, floofer, pupper, puppo is populated, other columns are populated with a value "None" . This should be represented as a Null value.
###### 8. Source column data is not very useful for the analysis. Delete this column.

#### image-predictions.tsv

###### 9. column names must be renamed to something more descriptive about the data it holds
###### 10. Some breed name values populated in p1_dog are starting with upper case and some are starting with lower case letter.


### Tidiness 

#### twitter-archive-enhanced.csv

###### 1. Dog breed value can be restructured to be stored in one column instead of four different columns.

#### image-predictions.tsv

###### 2.  Re structure  or drop the columns that are unnecessary. such as img_num,p2, p2_conf,p2_dog,p3,p3_conf,p3_dog 

###### 3. twitter_archive_data has 2356 rows and twitter_image_data has 2075 rows. After removing the retweets we need to merge the data in these datasets with tweet_counts to have the data in single data frame



# Clean

#### Define
1. Select all the rows that are retweets
2. Delete all rows that are retweets
3. select the rows that are tweet replies
4. Delete the rows that are tweet replies
3. Delete columns in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id , retweeted_status_user_id, retweeted_status_timestamp after removing the retweets

#### Code

In [79]:
# Create a copy of the data frame
# we do not want to loose the original data set
tweet_archive_data_clean = tweet_archive_data.copy()


In [80]:
tweet_archive_data_clean.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [81]:
# creating boolean series True for retweeted_status_id isnull 
# original tweets have retweeted_status_id as null
retweet_is_null = pd.isnull(tweet_archive_data_clean['retweeted_status_id'])
tweet_archive_data_clean = tweet_archive_data_clean[retweet_is_null]

In [82]:
# creating boolean series True for reply_status_id isnull 
# original tweets have reply_status_id as null
reply_is_null = pd.isnull(tweet_archive_data_clean['in_reply_to_status_id'])
tweet_archive_data_clean = tweet_archive_data_clean[reply_is_null]

In [83]:
# Drop columns in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,
# retweeted_status_user_id, retweeted_status_timestamp 

tweet_archive_data_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id',
                            'retweeted_status_id','retweeted_status_user_id',
                            'retweeted_status_timestamp'],inplace = True, axis=1)

#### Test

In [84]:
tweet_archive_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


#### Define
name column has majority of the rows ( 745) have None populated.There are also 109 rows that have invalid values for name with values like ( 'the','a','an','such' etc).
1. Gather the values in the name column that start with lowercase. These seem to be invalid values. Store them in a list.
2. Replace these values with null values
3. Replace None with Null values


#### Code

In [86]:
# Create a boolean series for name starting with lowercase letter
name_lower_case = tweet_archive_data_clean['name'].str[0].str.islower()

In [87]:
import numpy as np

In [88]:
# store these name values into an array
invalid_names = tweet_archive_data_clean[name_lower_case].name.values

In [89]:
invalid_names

array(['such', 'a', 'quite', 'quite', 'not', 'one', 'incredibly', 'a', 'a',
       'very', 'my', 'one', 'not', 'his', 'one', 'a', 'a', 'a', 'an',
       'very', 'actually', 'a', 'just', 'getting', 'mad', 'very', 'this',
       'unacceptable', 'all', 'a', 'old', 'a', 'infuriating', 'a', 'a',
       'a', 'an', 'a', 'a', 'very', 'getting', 'just', 'a', 'the', 'the',
       'actually', 'by', 'a', 'officially', 'a', 'the', 'the', 'a', 'a',
       'a', 'a', 'life', 'a', 'one', 'a', 'a', 'a', 'light', 'just',
       'space', 'a', 'the', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a',
       'an', 'a', 'the', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a',
       'a', 'quite', 'a', 'an', 'a', 'an', 'the', 'the', 'a', 'a', 'an',
       'a', 'a', 'a', 'a'], dtype=object)

In [90]:
# replace the above values in the name column with null values
tweet_archive_data_clean.name.replace(invalid_names,np.nan, inplace = True)

In [91]:
# replace the "None" value name column with null values
tweet_archive_data_clean.name.replace('None',np.nan, inplace = True)

#### Test

In [93]:
tweet_archive_data_clean.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', nan, 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver', 'Jim',
       'Zeke', 'Ralphus', 'Gerald', 'Jeffrey', 'Canela', 'Maya', 'Mingus',
       'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey', 'Earl', 'Lola',
       'Kevin', 'Yogi', 'Noah', 'Bella', 'Grizzwald', 'Rusty', 'Gus',
       'Stanley', 'Alfy', 'Koko', 'Rey', 'Gary', 'Elliot', 'Louis',
       'Jesse', 'Romeo', 'Bailey', 'Duddles', 'Jack', 'Steven', 'Beau',
       'Snoopy', 'Shadow', 'Emmy', 'Aja', 'Penny', 'Dante', 'Nelly',
       'Ginger', 'Benedict', 'Venti', 'Goose', 'Nugget', 'Cash', 'Jed',
       'Sebastian', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Boomer', 'Cody', 'Rumble', 'Clifford', 'Dewey',
       'Scout', 'Gizmo', 'Walter', 'Cooper', 'Harold', 'Shikha', 'Lili',
       'Jamesy', 'Coco', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Neptune', 'Belle', 'Quinn', 'Zooey', 'Dave', 'Jersey', 'Hobbes',


#### Define
1. Change the timestamp column to date data type

#### Code

In [98]:
tweet_archive_data_clean.timestamp = pd.to_datetime( tweet_archive_data_clean.timestamp,infer_datetime_format=True)

In [99]:
tweet_archive_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null datetime64[ns]
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  1390 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 293.0+ KB


#### Define
There are 23 rows where rating denominator is not equal to 10
 For consistency purposes I onyl want to keep the records where denominator is  equal to 10.

1. Filter out the rows where ratings denominator is not equal to 10

#### Code

In [100]:
tweet_archive_data_clean = tweet_archive_data_clean[tweet_archive_data_clean.rating_denominator == 10]

#### Test

In [101]:
tweet_archive_data_clean.rating_denominator.unique()

array([10])

#### Define
1. Filter out extreme outliers in Ratings numerator
i.e delete rows where ratings numerator is ( 1776,420,75) any ratings greater than equal to 74

In [102]:
tweet_archive_data_clean.rating_numerator.unique()

array([  13,   12,   14,    5,   11,    6,   10,    0,   75,   27,    3,
          7,    8,    9,    4, 1776,   26,    2,    1,  420])

#### Code

In [103]:
tweet_archive_data_clean = tweet_archive_data_clean[tweet_archive_data_clean.rating_numerator < 74]

#### Test

In [104]:
tweet_archive_data_clean.rating_numerator.unique()

array([13, 12, 14,  5, 11,  6, 10,  0, 27,  3,  7,  8,  9,  4, 26,  2,  1])

#### Define
Delete source and expanded_urls column

#### Code

In [105]:
tweet_archive_data_clean.drop(['source','expanded_urls'],inplace = True, axis=1)

#### Test

In [106]:
tweet_archive_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2077 non-null int64
timestamp             2077 non-null datetime64[ns]
text                  2077 non-null object
rating_numerator      2077 non-null int64
rating_denominator    2077 non-null int64
name                  1385 non-null object
doggo                 2077 non-null object
floofer               2077 non-null object
pupper                2077 non-null object
puppo                 2077 non-null object
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 178.5+ KB


##### Tidiness for twitter_archive_data

#### Define

1. Create a new columns called breed_type and populate it using the data in the columns doggo,floofer,pupper and puppo. 


#### Code

In [109]:
tweet_archive_data_clean[['doggo', 'floofer','pupper', 'puppo']].sample(5)

,doggo,floofer,pupper,puppo
796,None,None,None,None
1524,None,None,None,None
1409,None,None,None,None
985,doggo,None,None,None
1327,None,None,pupper,None


In [110]:
# replace None value with Null
columns = ['doggo', 'floofer','pupper', 'puppo']
for col in columns:
    tweet_archive_data_clean[col].replace('None',np.nan, inplace = True)

In [152]:
tweet_archive_data_clean.query('tweet_id in (855851453814013952,892420643555336193,890240255349198849)')

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,NaN,NaN,NaN,NaN
9,890240255349198849,2017-07-26 15:59:51,This is Cassie. She is a college pup. Studying...,14,10,Cassie,doggo,NaN,NaN,NaN
191,855851453814013952,2017-04-22 18:31:02,Here's a puppo participating in the #ScienceMa...,13,10,NaN,doggo,NaN,NaN,puppo


In [174]:
def breed_type(in_cols):
    breed = []
    for col in in_cols:
        if col in in_cols:
            breed.append(col)
    if len(breed)>0:
        return ','.join(breed)
    else:
        return None
    
    

In [176]:
test_df=tweet_archive_data_clean.query('tweet_id in (855851453814013952,892420643555336193,890240255349198849)')

In [177]:
test_df[columns].apply(breed_type,axis=1 )

0             None
9            doggo
191    doggo,puppo
dtype: object

In [178]:
tweet_archive_data_clean['breed'] = tweet_archive_data_clean[columns].apply(breed_type,axis=1 )

In [179]:
tweet_archive_data_clean[['breed','doggo', 'floofer','pupper', 'puppo']]

,breed,doggo,floofer,pupper,puppo
0,None,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN
3,None,NaN,NaN,NaN,NaN
4,None,NaN,NaN,NaN,NaN
5,None,NaN,NaN,NaN,NaN
6,None,NaN,NaN,NaN,NaN
7,None,NaN,NaN,NaN,NaN
8,None,NaN,NaN,NaN,NaN
9,doggo,doggo,NaN,NaN,NaN


In [180]:
tweet_archive_data_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], 
    inplace = True, axis = 1)

In [181]:
tweet_archive_data_clean.breed.unique()

array([None, 'doggo', 'puppo', 'pupper', 'floofer', 'doggo,puppo',
       'doggo,floofer', 'doggo,pupper'], dtype=object)

#### Test

In [182]:
tweet_archive_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2355
Data columns (total 7 columns):
tweet_id              2077 non-null int64
timestamp             2077 non-null datetime64[ns]
text                  2077 non-null object
rating_numerator      2077 non-null int64
rating_denominator    2077 non-null int64
name                  1385 non-null object
breed                 336 non-null object
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 129.8+ KB


#### Quality for  tweet_image_data

#### Define
1. Drop the columns img_num,p1_dog,p2, p2_conf, p2_dog, p3, p3_conf,p3_dog
2. Rename the columns jpg_url, p1, p1_conf and p1_dog to descriptive column names

#### Code

In [183]:
# Create a copy of the data frame
# we do not want to update the original data set
tweet_image_data_clean = tweet_image_data.copy()



In [184]:
tweet_image_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [185]:
tweet_image_data_clean.drop(['img_num','p1_dog','p2', 'p2_conf', 'p2_dog', 'p3', 
                  'p3_conf', 'p3_dog'], inplace = True, axis = 1)

In [186]:
tweet_image_data_clean.rename(columns = {'jpg_url': 'image_url', 
                               'p1': 'prediction', 
                               'p1_conf': 'confidence_level'}, inplace = True)

In [187]:
tweet_image_data_clean.head()

,tweet_id,image_url,prediction,confidence_level
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,miniature_pinscher,0.560311


#### Tidiness
Join the three data sets into one single data frame

In [188]:
tweet_archive_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2355
Data columns (total 7 columns):
tweet_id              2077 non-null int64
timestamp             2077 non-null datetime64[ns]
text                  2077 non-null object
rating_numerator      2077 non-null int64
rating_denominator    2077 non-null int64
name                  1385 non-null object
breed                 336 non-null object
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 129.8+ KB


In [189]:
tweet_image_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 4 columns):
tweet_id            2075 non-null int64
image_url           2075 non-null object
prediction          2075 non-null object
confidence_level    2075 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 64.9+ KB


In [190]:
tweet_count_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


#### Code

In [191]:
# Merge the three data frames
twitter_archive_master = tweet_archive_data_clean.merge(tweet_count_data,
                                                   on = 'tweet_id', how = 'inner')
twitter_archive_master = twitter_archive_master.merge(tweet_image_data_clean, 
                                            on = 'tweet_id', how = 'left')

##### Test

In [192]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id              2075 non-null int64
timestamp             2075 non-null datetime64[ns]
text                  2075 non-null object
rating_numerator      2075 non-null int64
rating_denominator    2075 non-null int64
name                  1383 non-null object
breed                 336 non-null object
retweet_count         2075 non-null int64
favorite_count        2075 non-null int64
image_url             1949 non-null object
prediction            1949 non-null object
confidence_level      1949 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 210.7+ KB


In [193]:
twitter_archive_master.query("tweet_id in ('890729181411237888','890240255349198849')")

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,breed,retweet_count,favorite_count,image_url,prediction,confidence_level
7,890729181411237888,2017-07-28 00:22:40,When you watch your owner call another dog a g...,13,10,NaN,None,18341,63867,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,Pomeranian,0.566142
9,890240255349198849,2017-07-26 15:59:51,This is Cassie. She is a college pup. Studying...,14,10,Cassie,doggo,7172,31193,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,Pembroke,0.511319


In [194]:
tweet_image_data_clean.query("tweet_id in ('890729181411237888','890240255349198849')")

,tweet_id,image_url,prediction,confidence_level
2065,890240255349198849,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,Pembroke,0.511319
2067,890729181411237888,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,Pomeranian,0.566142


In [195]:
tweet_count_data.query("tweet_id in ('890729181411237888','890240255349198849')")

,tweet_id,retweet_count,favorite_count
7,890729181411237888,18341,63867
9,890240255349198849,7172,31193


# Store 

In [196]:
# Store tables to files
tweet_count_data.to_csv('tweet_count_data_clean.csv', encoding='utf-8', index=False)
tweet_image_data_clean.to_csv('tweet_image_data_clean.csv', encoding='utf-8', index=False)
tweet_archive_data_clean.to_csv('tweet_archive_data_clean.csv', encoding='utf-8', index=False)
twitter_archive_master.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

### Code for data Analysis and Visualization is in act_report.ipynb